In [1]:
# Imports
import numpy as np
import spikeinterface as si

In [30]:
# Properties of the in-memory dataset
num_channels=7
samplerate=30000
duration=20
num_timepoints=int(samplerate*duration)

In [31]:
# Generate a pure-noise timeseries dataset and a linear geometry
timeseries=np.random.normal(0,10,(num_channels,num_timepoints))
geom=np.zeros((num_channels,2))
geom[:,0]=range(num_channels)

In [32]:
# Define the in-memory recording extractor
RX=si.NumpyRecordingExtractor(timeseries=timeseries,geom=geom,samplerate=samplerate)

In [33]:
# Demonstrate the API for extracting information
print('Num. channels = {}'.format(len(RX.getChannelIds())))
print('Sampling frequency = {} Hz'.format(RX.getSamplingFrequency()))
print('Num. timepoints = {}'.format(RX.getNumFrames()))
print('Stdev. on third channel = {}'.format(np.std(RX.getTraces(channel_ids=2))))
print('Location of third electrode = {}'.format(RX.getChannelProperty(channel_id=2, property_name='location')))

Num. channels = 7
Sampling frequency = 30000.0 Hz
Num. timepoints = 600000
Stdev. on third channel = 10.005907742354227
Location of third electrode = [2. 0.]


In [34]:
# Write this dataset in the MountainSort format
si.MdaRecordingExtractor.writeRecording(recording=RX,save_path='sample_mountainsort_dataset')

In [35]:
# Read this dataset with the Mda recording extractor
RX2=si.MdaRecordingExtractor(dataset_directory='sample_mountainsort_dataset')

In [36]:
# We should get he same information as above
print('Num. channels = {}'.format(len(RX2.getChannelIds())))
print('Sampling frequency = {} Hz'.format(RX2.getSamplingFrequency()))
print('Num. timepoints = {}'.format(RX2.getNumFrames()))
print('Stdev. on third channel = {}'.format(np.std(RX2.getTraces(channel_ids=2))))
print('Location of third electrode = {}'.format(RX.getChannelProperty(channel_id=4, property_name='location')))

Num. channels = 7
Sampling frequency = 30000.0 Hz
Num. timepoints = 600000
Stdev. on third channel = 10.005908012390137
Location of third electrode = [4. 0.]


In [37]:
#Putting Epochs into our recording (Adding a name to a time period in the recording)
RX2.addEpoch(epoch_name='stimulation', start_frame=1000, end_frame=6000)
RX2.addEpoch(epoch_name='post_stimulation', start_frame=6000, end_frame=10000)
RX2.addEpoch(epoch_name='pre_stimulation', start_frame=0, end_frame=1000)

In [38]:
#Check Epoch names in our recording
RX2.getEpochNames()

['pre_stimulation', 'stimulation', 'post_stimulation']

In [39]:
#Return a SubRecordingExtractor that is a view to our epoch. Can view info about it in parent extractor
RX3 = RX2.getEpoch(epoch_name='stimulation')
epoch_info = RX2.getEpochInfo('stimulation')
start_frame = epoch_info['start_frame']
end_frame = epoch_info['end_frame']

print('Epoch Name = stimulation')
print('Start Frame = {}'.format(start_frame))
print('End Frame = {}'.format(end_frame))
print('Mean. on second channel during stimulation = {}'.format(np.mean(RX3.getTraces(channel_ids=1))))
print('Location of third electrode = {}'.format(RX.getChannelProperty(channel_id=2, property_name='location')))

Epoch Name = stimulation
Start Frame = 1000
End Frame = 6000
Mean. on second channel during stimulation = 0.08312392234802246
Location of third electrode = [2. 0.]


In [42]:
# Can extract an arbitrary subset of your data/channels manually without epoch functionality
RX4=si.SubRecordingExtractor(parent_recording=RX2,channel_ids=[2,3,4,5],start_frame=14000,
                             end_frame=16000)

In [43]:
# Show the information for this sub-dataset
print('Num. channels = {}'.format(len(RX4.getChannelIds())))
print('Sampling frequency = {} Hz'.format(RX4.getSamplingFrequency()))
print('Num. timepoints = {}'.format(RX4.getNumFrames()))
print('Stdev. on third channel = {}'.format(np.std(RX4.getTraces(channel_ids=2))))
print('Location of third electrode = {}'.format(RX4.getChannelProperty(channel_id=2, property_name='location')))

Num. channels = 4
Sampling frequency = 30000.0 Hz
Num. timepoints = 2000
Stdev. on third channel = 9.99359130859375
Location of third electrode = [2. 0.]


In [44]:
#Can rename the sub-dataset channel ids (Channel id mapping: 0-->2, 1-->3, 2-->4, 3-->5)
RX5=si.SubRecordingExtractor(parent_recording=RX2,channel_ids=[2,3,4,5], 
                             renamed_channel_ids = [0,1,2,3],
                             start_frame=14000,end_frame=16000)
print('New ids = {}'.format(RX5.getChannelIds()))
print('Original ids = {}'.format(RX5.getOriginalChannelIds([0,1,2,3])))

New ids = [0, 1, 2, 3]
Original ids = [2, 3, 4, 5]


In [45]:
# Show the information for this sub-dataset 
print('Num. channels = {}'.format(len(RX5.getChannelIds())))
print('Sampling frequency = {} Hz'.format(RX5.getSamplingFrequency()))
print('Num. timepoints = {}'.format(RX5.getNumFrames()))
print('Stdev. on third channel = {}'.format(np.std(RX5.getTraces(channel_ids=0))))
print('Location of third electrode = {}'.format(RX5.getChannelProperty(channel_id=0, property_name='location')))

Num. channels = 4
Sampling frequency = 30000.0 Hz
Num. timepoints = 2000
Stdev. on third channel = 9.99359130859375
Location of third electrode = [2. 0.]
